# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:
- Practice your SQL knowledge


## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

In [3]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('orderdetails',), ('payments',), ('offices',), ('customers',), ('orders',), ('productlines',), ('products',), ('employees',)]


In [4]:
c.execute("PRAGMA table_info(customers)").fetchall()

[(0, 'customerNumber', '', 0, None, 0),
 (1, 'customerName', '', 0, None, 0),
 (2, 'contactLastName', '', 0, None, 0),
 (3, 'contactFirstName', '', 0, None, 0),
 (4, 'phone', '', 0, None, 0),
 (5, 'addressLine1', '', 0, None, 0),
 (6, 'addressLine2', '', 0, None, 0),
 (7, 'city', '', 0, None, 0),
 (8, 'state', '', 0, None, 0),
 (9, 'postalCode', '', 0, None, 0),
 (10, 'country', '', 0, None, 0),
 (11, 'salesRepEmployeeNumber', '', 0, None, 0),
 (12, 'creditLimit', '', 0, None, 0)]

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, and address for all customers in California with a credit limit greater than 25000.00. 

In [13]:
c.execute("""

SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit 
FROM customers WHERE state == 'CA' 
and CreditLimit > 25000.00

""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500.00
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600.00
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.00
3,Julie,Young,6265557265,78934 Hillside Dr.,90700.00
4,Mary,Young,3105552373,4097 Douglas Av.,11000.00
5,Valarie,Thompson,7605558146,361 Furth Circle,105000.00
6,Julie,Brown,6505551386,7734 Strong St.,105000.00
7,Brian,Chandler,2155554369,6047 Douglas Av.,57700.00
8,Sue,Frick,4085553659,3086 Ingle Ln.,77600.00
9,Steve,Thompson,3105553722,3675 Furth Circle,55400.00


In [20]:
c.execute("""SELECT contactFirstName, contactLastName, phone, addressline1, creditLimit from customers
             WHERE CAST (creditLimit AS DECIMAL) > 25000.00 and state = 'CA' 
             ORDER BY CAST (creditLimit AS DECIMAL(6,2)) DESC
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500.00
1,Valarie,Thompson,7605558146,361 Furth Circle,105000.00
2,Julie,Brown,6505551386,7734 Strong St.,105000.00
3,Julie,Young,6265557265,78934 Hillside Dr.,90700.00
4,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.00
5,Sue,Frick,4085553659,3086 Ingle Ln.,77600.00
6,Julie,Murphy,6505555787,5557 North Pendale Street,64600.00
7,Sue,Taylor,4155554312,2793 Furth Circle,60300.00
8,Brian,Chandler,2155554369,6047 Douglas Av.,57700.00
9,Steve,Thompson,3105553722,3675 Furth Circle,55400.00


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [8]:
df=pd.read_sql_query("""

SELECT AVG(creditLimit), state FROM customers GROUP BY state ORDER BY 1 ASC LIMIT 5

;""", con=conn)
df.head()

,AVG(creditLimit),state
0,0.0,Co. Cork
1,0.0,Pretoria
2,43000.0,NJ
3,48700.0,Québec
4,51600.0,Queensland


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [7]:
df=pd.read_sql_query("""

SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, 
od.quantityOrdered
FROM customers c
JOIN orders o using(customerNumber)
JOIN orderdetails od using(orderNumber)

;""", con=conn)
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/expected-output-3.png'>

## HAVING and ORDER BY

Now, repeat the last query, but only get orders from customers that have a quantityOrdered value greater than 30. Sort the rows in ascending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [50]:
df=pd.read_sql_query("""

SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, 
od.quantityOrdered
FROM customers c
JOIN orders o using(customerNumber)
JOIN orderdetails od using(orderNumber)
GROUP BY quantityOrdered
HAVING SUM(od.quantityOrdered >30)
ORDER BY quantityOrdered ASC

;""", con=conn)
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,"Extreme Desk Decorations, Ltd",412,10418,Shipped,10
1,Tekni Collectables Inc.,328,10401,On Hold,11
2,Salzburg Collectables,382,10419,Shipped,12
3,The Sharp Gifts Warehouse,450,10407,On Hold,13
4,"Tokyo Collectables, Ltd",398,10408,Shipped,15


#### Expected Output

<img src='images/expected-output-4.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [37]:
c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered FROM Customers c JOIN Orders o 
ON c.customerNumber = o.customerNumber JOIN OrderDetails od ON od.orderNumber = o.orderNumber
GROUP BY od.quantityOrdered
HAVING SUM(od.quantityOrdered) > 10 
ORDER BY od.quantityOrdered ASC""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,"Extreme Desk Decorations, Ltd",412,10418,Shipped,10
1,Tekni Collectables Inc.,328,10401,On Hold,11
2,Salzburg Collectables,382,10419,Shipped,12
3,The Sharp Gifts Warehouse,450,10407,On Hold,13
4,"Tokyo Collectables, Ltd",398,10408,Shipped,15


In [51]:
c.execute("""select lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN (SELECT officeCode 
                                                FROM offices 
                                                JOIN employees
                                                USING(officeCode)
                                                GROUP BY 1
                                                HAVING COUNT(employeeNumber) < 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!